In [1]:
import logging, logging.handlers
import sys
import pickle
import numpy as np
from collections import namedtuple
from itertools import count
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

import seaborn as sns; sns.set(color_codes=True)
%matplotlib inline
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import math

# round_ = 1
# lookahead = 1
target_update = 10000
w = 10
h = 10
life_cost = 0.05
# env_name = 'Grid'
# env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
# logger = logging.getLogger()
# file_name = './data/results_%s.log' %(env_name)
# fh = logging.handlers.RotatingFileHandler(file_name)
# fh.setLevel(logging.DEBUG)#no matter what level I set here
# formatter = logging.Formatter('%(asctime)s:%(message)s')
# fh.setFormatter(formatter)
# logger.addHandler(fh)

In [2]:
batchsize = 64
USE_CUDA = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
seed = 0
number_action = 4
dimension = 2

import torch
import torch.nn as nn
import torch.nn.functional as F
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv' or 'SNConv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
class DQN(nn.Module):
    def __init__(self, dimension=2, num_actions=number_action):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(dimension, 50)
#         self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(50, num_actions)
    def forward(self, x):
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
#         x = F.relu(self.fc2(x.view(x.size(0), -1)))
        return self.fc3(x)


In [3]:
w_ = w-1
h_ = 0
h__ = h-1
gamma = 0.99
pos_rew = 1
def recurs(imagine_,h,Q):
    if h == 0:
        val = Q(imagine_.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)
        return val[0], val[1][0]
    else:
        action = 0
        for i in range(number_action):
            imagine = torch.clone(imagine_)
            if i == 0:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] - 1,0,h-1))
                cum_return_ = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                               - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                
            if i == 1:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] - 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 1
            if i == 2:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] + 1,0,h-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 2
            if i == 3:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] + 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 3   
#         print(cum_return_)            
        return cum_return_, torch.tensor(action)
                         

In [ ]:
#action
#  0
#1   3
#  2
#         w
# x=0,y=0    x=w,y=0    
#                      h
# x=0,y=h    x=w,y=h   

for round_count in range(10):
    round_ = round_count + 1
    lookahead = 0
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    init_w = 0
    init_h = h-1
    tot_epis = 1000
    epsilon = .1


    Q = DQN(dimension,number_action).type(dtype) 
    Target_Q = DQN(dimension,number_action).type(dtype) 
    Q.apply(weights_init)
    optimizer = optim.SGD(Q.parameters(), lr = .0001, momentum=0.9)


    actions = torch.zeros((1),dtype=torch.int8)
    states = torch.zeros((1,2),dtype=torch.int8)
    # done = torch.zeros((1)).type(dtype) 
    # states = torch.tensor([init_w,init_h])
    rewards = torch.zeros((1)).type(dtype) 
    tot_rew = torch.zeros((1)).type(dtype) 
    # next_states = torch.zeros((1,2),dtype=torch.int8)
    e = 0
    tot_time = 2100000
    time = 0
    while time < tot_time:
        t = 0
        goal = 0
        cum_rew = 0
        # Initiate the state
        if time > 0:
            states = torch.cat((states,torch.tensor([[0,0]],dtype=torch.int8)),0)
        state = torch.clone(states[-1])
        while t<100 and goal==0:
            time = time+1
            next_state =  torch.clone(state)
            sample = random.random()
            if sample < epsilon:
                action = torch.tensor(random.randrange(number_action))
            else:
                action = recurs(state,lookahead,Q)[1]
            if time == 1:
                actions[0] = action.unsqueeze(0).type(dtype) 
            else:
                actions = torch.cat((actions,action.unsqueeze(0).type(torch.int8)),0)
            reward = pos_rew*torch.all(torch.eq(state, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                    - torch.all(torch.eq(state[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
    #         print(state)
            if reward != -1 * life_cost :
                goal = 1
#                 if reward == pos_rew - life_cost:
#                     print('bingo:t:',t,state)

            if time == 1:
                rewards[0] = reward.unsqueeze(0)
            else:
                rewards = torch.cat((rewards,reward.unsqueeze(0)),0)
            if action == 0:
                next_state[1] = torch.clone(torch.clamp(state[1] - 1,0,h-1))
            if action == 1:
                next_state[0] = torch.clone(torch.clamp(state[0] - 1,0,w-1))
            if action == 2:
                next_state[1] = torch.clone(torch.clamp(state[1] + 1,0,h-1))
            if action == 3:
                next_state[0] = torch.clone(torch.clamp(state[0] + 1,0,w-1))
    #         print(state.type(torch.int).numpy(),action.type(torch.int).numpy(),next_state.type(torch.int).numpy()) 
            cum_rew = cum_rew + gamma * torch.clone(reward)   
            states = torch.cat((states,torch.clone(next_state.unsqueeze(0))),0)
            state = torch.clone(next_state)


    #         current_value = Q(state.unsqueeze(0).unsqueeze(0).type(dtype))[0][action]    
    #         loss = current_value-reward.type(dtype)-(1-goal)*gamma*Target_Q(next_state.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)[0].detach()
    #         d_error = loss.clamp(-1, 1)[0]
    #         optimizer.zero_grad()
    #         current_value.backward(d_error.data)
    #         optimizer.step()
            if time>5000:
                if time % 4 == 0:        
                    choices = np.random.choice(time, batchsize)
                    current_action = actions[choices].unsqueeze(1)
                    current_rewards = rewards[choices].type(dtype)
                    indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
                    current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
                    current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
                    loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
                    d_error = loss.clamp(-1, 1)
                    optimizer.zero_grad()
                    current_value.backward(d_error.data)
                    optimizer.step()
            t = t+1

            if time % target_update == 0:
                Target_Q.load_state_dict(Q.state_dict())


        if len(tot_rew) == 1:
            tot_rew[0] = cum_rew.unsqueeze(0).type(dtype)
        else:
            tot_rew = torch.cat((tot_rew,cum_rew.unsqueeze(0).type(dtype)),0) 
        if e%100==0:    
            print('e:',e,'t:',time,'rew:',cum_rew.type(dtype).numpy())
        e = e + 1
    #     s = torch.zeros((1,2),dtype=torch.int32)
    #     s[0,0] = 3
    #     s[0,1] = 3
    #     ss = torch.zeros((1,2),dtype=torch.int32)
    #     ss[0,0] = 0
    #     ss[0,1] = 0
    #     print(Q(s.type(dtype))-Q(ss.type(dtype)))


    fnam = './data/reward%s' %(env_name)
    np.save(fnam,rewards.numpy())                          




    #                 for i2 in range(number_action):







    #  0
    #1   3
    #  2 

e: 0 t: 100 rew: -4.9499993
e: 100 t: 8455 rew: -4.9499993
e: 200 t: 13527 rew: -5.7419996
e: 300 t: 18137 rew: -2.5245
e: 400 t: 22306 rew: -1.8314998
e: 500 t: 25985 rew: -0.8414997
e: 600 t: 30787 rew: -4.9004993
e: 700 t: 35606 rew: 0.49500006
e: 800 t: 40890 rew: 0.49500006
e: 900 t: 46525 rew: -3.8609993
e: 1000 t: 52678 rew: -1.6829998
e: 1100 t: 58662 rew: -4.9499993
e: 1200 t: 65542 rew: -3.5639997
e: 1300 t: 72622 rew: -2.5244994
e: 1400 t: 79493 rew: -4.9499993
e: 1500 t: 86069 rew: -4.9499993
e: 1600 t: 93286 rew: -4.9499993
e: 1700 t: 100214 rew: -5.494499
e: 1800 t: 106993 rew: -3.0689995
e: 1900 t: 114011 rew: -4.9499993
e: 2000 t: 120782 rew: -4.9499993
e: 2100 t: 126922 rew: -4.9499993
e: 2200 t: 133956 rew: -2.5244994
e: 2300 t: 140508 rew: -4.9499993
e: 2400 t: 147137 rew: -1.6829998
e: 2500 t: 154192 rew: -0.7919997
e: 2600 t: 160236 rew: 0.3960001
e: 2700 t: 167253 rew: -4.9499993
e: 2800 t: 174074 rew: -4.9499993
e: 2900 t: 180939 rew: -4.9499993
e: 3000 t: 187676

e: 23900 t: 1013094 rew: -1.4849999
e: 24000 t: 1014840 rew: -1.8314998
e: 24100 t: 1017256 rew: -4.9499993
e: 24200 t: 1019249 rew: -1.5344999
e: 24300 t: 1021129 rew: -1.4849999
e: 24400 t: 1023382 rew: -1.4849999
e: 24500 t: 1025666 rew: -4.9499993
e: 24600 t: 1027603 rew: -1.5839999
e: 24700 t: 1029765 rew: -1.6334999
e: 24800 t: 1031511 rew: -1.4849999
e: 24900 t: 1033739 rew: -1.4849999
e: 25000 t: 1036000 rew: -1.5839999
e: 25100 t: 1038112 rew: -1.5839999
e: 25200 t: 1040001 rew: -1.5344999
e: 25300 t: 1041869 rew: -2.7719998
e: 25400 t: 1043815 rew: -1.5344999
e: 25500 t: 1046348 rew: -1.6829998
e: 25600 t: 1048308 rew: -1.4849999
e: 25700 t: 1050786 rew: -1.6829998
e: 25800 t: 1052739 rew: -1.4849999
e: 25900 t: 1054362 rew: -1.5839999
e: 26000 t: 1056211 rew: -1.4849999
e: 26100 t: 1058359 rew: -1.8314998
e: 26200 t: 1060317 rew: -1.4849999
e: 26300 t: 1062087 rew: -1.4849999
e: 26400 t: 1063830 rew: -2.3265
e: 26500 t: 1065896 rew: -1.4849999
e: 26600 t: 1067670 rew: -1.732

e: 46800 t: 1425024 rew: -1.4849999
e: 46900 t: 1426778 rew: -1.5839999
e: 47000 t: 1428315 rew: -4.5044994
e: 47100 t: 1430161 rew: -1.5839999
e: 47200 t: 1431868 rew: -1.4849999
e: 47300 t: 1433398 rew: -1.4849999
e: 47400 t: 1435045 rew: -1.6334999
e: 47500 t: 1436601 rew: -1.4849999
e: 47600 t: 1437944 rew: -1.7819998
e: 47700 t: 1439433 rew: -1.4849999
e: 47800 t: 1441090 rew: -1.4849999
e: 47900 t: 1442723 rew: -2.0294998
e: 48000 t: 1444140 rew: -1.5344999
e: 48100 t: 1446065 rew: -1.5344999
e: 48200 t: 1447832 rew: -1.4849999
e: 48300 t: 1449527 rew: -1.5839999
e: 48400 t: 1450931 rew: -1.5344999
e: 48500 t: 1452617 rew: -1.5344999
e: 48600 t: 1454239 rew: -4.9499993
e: 48700 t: 1455701 rew: -1.5344999
e: 48800 t: 1457175 rew: -1.6334999
e: 48900 t: 1458845 rew: -1.5344999
e: 49000 t: 1460436 rew: -5.692499
e: 49100 t: 1461880 rew: -1.4849999
e: 49200 t: 1463479 rew: -1.4849999
e: 49300 t: 1464890 rew: -1.5344999
e: 49400 t: 1466662 rew: -1.6334999
e: 49500 t: 1467998 rew: -1.4

e: 69600 t: 1761187 rew: -1.4849999
e: 69700 t: 1762575 rew: -1.7324998
e: 69800 t: 1763911 rew: -1.5839999
e: 69900 t: 1765180 rew: -1.4849999
e: 70000 t: 1766680 rew: -1.5839999
e: 70100 t: 1768231 rew: -1.5344999
e: 70200 t: 1769660 rew: -1.6829998
e: 70300 t: 1770937 rew: -1.5344999
e: 70400 t: 1772260 rew: -1.4849999
e: 70500 t: 1773596 rew: -1.4849999
e: 70600 t: 1774904 rew: -5.593499
e: 70700 t: 1776320 rew: -1.4849999
e: 70800 t: 1777586 rew: -1.5839999
e: 70900 t: 1778924 rew: -1.5344999
e: 71000 t: 1780211 rew: -1.6334999
e: 71100 t: 1781717 rew: -1.4849999
e: 71200 t: 1782978 rew: -1.4849999
e: 71300 t: 1784275 rew: -1.5344999
e: 71400 t: 1785610 rew: -1.4849999
e: 71500 t: 1787094 rew: -1.4849999
e: 71600 t: 1788373 rew: -1.5344999
e: 71700 t: 1789857 rew: -1.5344999
e: 71800 t: 1791212 rew: -1.4849999
e: 71900 t: 1792494 rew: -1.6829998
e: 72000 t: 1793857 rew: -1.4849999
e: 72100 t: 1795478 rew: -1.4849999
e: 72200 t: 1796748 rew: -1.5344999
e: 72300 t: 1798113 rew: -1.4

e: 92400 t: 2080659 rew: -1.4849999
e: 92500 t: 2082122 rew: -1.4849999
e: 92600 t: 2083524 rew: -1.5344999
e: 92700 t: 2084759 rew: -1.5344999
e: 92800 t: 2086456 rew: -1.4849999
e: 92900 t: 2087959 rew: -1.4849999
e: 93000 t: 2089272 rew: -1.6334999
e: 93100 t: 2090583 rew: -1.5344999
e: 93200 t: 2091915 rew: -1.4849999
e: 93300 t: 2093385 rew: -1.5344999
e: 93400 t: 2094820 rew: -2.0294998
e: 93500 t: 2096049 rew: 0.44550008
e: 93600 t: 2097358 rew: -1.4849999
e: 93700 t: 2098922 rew: -1.4849999
e: 0 t: 100 rew: -4.9499993
e: 100 t: 10100 rew: -4.9499993
e: 200 t: 17317 rew: -4.9499993
e: 300 t: 22980 rew: -0.3959998
e: 400 t: 29118 rew: -4.1084995
e: 500 t: 34871 rew: -4.9499993
e: 600 t: 39834 rew: -4.9499993
e: 700 t: 46480 rew: 0.2970001
e: 800 t: 52996 rew: -2.0294998
e: 900 t: 57495 rew: 0.49500006
e: 1000 t: 62894 rew: 0.49500006
e: 1100 t: 67778 rew: 0.3960001
e: 1200 t: 73609 rew: -2.574
e: 1300 t: 79157 rew: 0.49500006
e: 1400 t: 83981 rew: 0.49500006
e: 1500 t: 88741 rew:

...

e: 77400 t: 1899275 rew: 0.49500006
e: 77500 t: 1901392 rew: 0.49500006
e: 77600 t: 1903283 rew: 0.3960001
e: 77700 t: 1906232 rew: 0.44550008
e: 77800 t: 1908795 rew: 0.49500006
e: 77900 t: 1911863 rew: 0.49500006
e: 78000 t: 1915894 rew: 0.3465001
e: 78100 t: 1919300 rew: -2.4749994
e: 78200 t: 1922764 rew: -3.4649994
e: 78300 t: 1926796 rew: 0.49500006
e: 78400 t: 1930345 rew: -4.9499993
e: 78500 t: 1934968 rew: -0.19799984
e: 78600 t: 1937525 rew: -4.9499993
e: 78700 t: 1939937 rew: 0.49500006
e: 78800 t: 1944012 rew: -0.2969998
e: 78900 t: 1946407 rew: -0.3959998
e: 79000 t: 1948608 rew: 0.3960001
e: 79100 t: 1951347 rew: 0.49500006
e: 79200 t: 1954275 rew: -1.6334999
e: 79300 t: 1956564 rew: 0.3465001
e: 79400 t: 1958605 rew: 0.49500006
e: 79500 t: 1961381 rew: -4.9499993
e: 79600 t: 1963732 rew: 0.49500006
e: 79700 t: 1967035 rew: 0.44550008
e: 79800 t: 1969192 rew: 0.49500006
e: 79900 t: 1971896 rew: 0.49500006
e: 80000 t: 1975161 rew: 0.49500006
e: 80100 t: 1976472 rew: 0.4950

e: 15900 t: 717084 rew: -4.9499993
e: 16000 t: 720106 rew: 0.49500006
e: 16100 t: 723355 rew: 0.49500006
e: 16200 t: 726657 rew: 0.44550008
e: 16300 t: 729265 rew: -3.0194993
e: 16400 t: 733104 rew: -4.9499993
e: 16500 t: 736299 rew: 0.44550008
e: 16600 t: 739885 rew: -3.8114996
e: 16700 t: 743574 rew: 0.2970001
e: 16800 t: 747187 rew: -1.0394996
e: 16900 t: 751018 rew: -1.3859997
e: 17000 t: 755082 rew: -0.2969998
e: 17100 t: 758914 rew: -4.9499993
e: 17200 t: 762081 rew: 0.49500006
e: 17300 t: 765107 rew: -1.4354997
e: 17400 t: 768928 rew: -0.49499977
e: 17500 t: 773249 rew: -0.44549978
e: 17600 t: 777251 rew: 0.49500006
e: 17700 t: 780713 rew: 0.49500006
e: 17800 t: 784772 rew: -3.8609993
e: 17900 t: 787370 rew: 0.44550008
e: 18000 t: 790207 rew: 1.7881393e-07
e: 18100 t: 793841 rew: 0.3960001
e: 18200 t: 797601 rew: 0.49500006
e: 18300 t: 802287 rew: 0.3960001
e: 18400 t: 805542 rew: -0.14849985
e: 18500 t: 809462 rew: 0.49500006
e: 18600 t: 812076 rew: 0.49500006
e: 18700 t: 81547

e: 39000 t: 1485079 rew: -3.0194993
e: 39100 t: 1488781 rew: 0.44550008
e: 39200 t: 1492175 rew: 0.099000156
e: 39300 t: 1495523 rew: -4.9499993
e: 39400 t: 1497712 rew: -1.6829998
e: 39500 t: 1500507 rew: 0.3960001
e: 39600 t: 1504066 rew: 0.49500006
e: 39700 t: 1506953 rew: -4.9499993
e: 39800 t: 1510393 rew: -2.4255
e: 39900 t: 1513354 rew: 0.44550008
e: 40000 t: 1516293 rew: 0.49500006
e: 40100 t: 1518820 rew: -4.2569995
e: 40200 t: 1522722 rew: -3.4154997
e: 40300 t: 1525725 rew: -4.8014994
e: 40400 t: 1529313 rew: 0.49500006
e: 40500 t: 1532379 rew: -4.9499993
e: 40600 t: 1535947 rew: -4.9499993
e: 40700 t: 1538813 rew: -4.9499993
e: 40800 t: 1541906 rew: -1.9799998
e: 40900 t: 1545233 rew: -4.4549994
e: 41000 t: 1549077 rew: 0.49500006
e: 41100 t: 1551358 rew: 0.49500006
e: 41200 t: 1553725 rew: 0.3960001
e: 41300 t: 1557562 rew: 0.3960001
e: 41400 t: 1559840 rew: 0.49500006
e: 41500 t: 1562556 rew: -1.6829998
e: 41600 t: 1565065 rew: -3.2669995
e: 41700 t: 1567150 rew: -1.83149

e: 61900 t: 2048350 rew: -1.8809998
e: 62000 t: 2050471 rew: -2.0294998
e: 62100 t: 2052194 rew: 0.3960001
e: 62200 t: 2054130 rew: -1.5839999
e: 62300 t: 2056328 rew: -1.5839999
e: 62400 t: 2058865 rew: -2.0294998
e: 62500 t: 2060991 rew: -1.6334999
e: 62600 t: 2063339 rew: -1.7819998
e: 62700 t: 2065061 rew: -3.2174997
e: 62800 t: 2066497 rew: -1.6334999
e: 62900 t: 2068501 rew: -1.6334999
e: 63000 t: 2070119 rew: -1.5839999
e: 63100 t: 2071734 rew: -4.1084995
e: 63200 t: 2073406 rew: -1.6334999
e: 63300 t: 2075090 rew: 0.49500006
e: 63400 t: 2076684 rew: -1.7819998
e: 63500 t: 2078329 rew: 0.49500006
e: 63600 t: 2079835 rew: -1.6829998
e: 63700 t: 2081468 rew: -1.7324998
e: 63800 t: 2082919 rew: -2.2274995
e: 63900 t: 2084444 rew: 0.3960001
e: 64000 t: 2085817 rew: -3.3659997
e: 64100 t: 2087358 rew: -1.7324998
e: 64200 t: 2089050 rew: 0.49500006
e: 64300 t: 2090676 rew: 0.49500006
e: 64400 t: 2092011 rew: -0.24749982
e: 64500 t: 2093941 rew: -1.6829998
e: 64600 t: 2095592 rew: -1.5

e: 20900 t: 792932 rew: 0.49500006
e: 21000 t: 795898 rew: -4.9499993
e: 21100 t: 799581 rew: -4.9499993
e: 21200 t: 802178 rew: 0.49500006
e: 21300 t: 805161 rew: -4.9499993
e: 21400 t: 808648 rew: -4.9499993
e: 21500 t: 811655 rew: -1.5344999
e: 21600 t: 814806 rew: -4.9499993
e: 21700 t: 818731 rew: -1.3859997
e: 21800 t: 822601 rew: -4.9499993


In [ ]:
bandwidth = 100
sampling = 1000
# tot_time = 100000

LA0 = np.zeros((2,int(tot_time/sampling )-bandwidth))
lookahead = 0
for r in range(2):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA0[r,:] = total_3


In [ ]:
LA1 = np.zeros((1,int(tot_time/sampling )-bandwidth))
lookahead = 1
for r in range(1):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA1[r,:] = total_3



In [ ]:
fonts = 15
fram = np.linspace(1,tot_time,int(tot_time/sampling ),dtype=np.int32)[0:-bandwidth]
plt.ticklabel_format(style='sci', axis='x')
plt.ticklabel_format(axis='both', style='sci', scilimits=(-2,2))
plt.legend(fontsize=fonts)

# ax = sns.tsplot(data=LA0,time=fram,value='Value',color = 'b',condition = "LA = 0")
ax = sns.tsplot(data=LA1,time=fram,value='Value',color = 'c',condition = "LA = 1")

plt.xlabel("Number of steps",fontsize=fonts, family = 'serif')
plt.ylabel("Average reward per episode",fontsize=fonts, family = 'serif')
plt.title("%s" %(env_name),fontsize=fonts, family = 'serif')
# ax.set(xlabel='Number of steps', ylabel='Average reward per episode')

# plt.savefig("Pong_reward_lambda_3_runs.png",format='png', dpi=2000)
# plt.savefig("Pong_reward_lambda_3_runs.eps",format='eps', dpi=2000)

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
ss = torch.zeros((1,2),dtype=torch.int32)
ss[0,0] = 0
ss[0,1] = 0
print(Q(s.type(dtype))-Q(ss.type(dtype)))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
Q(s.type(dtype))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 0
s[0,1] = 0
Q(s.type(dtype))

In [ ]:
current_nextstates= states[choices+1].unsqueeze(1).type(dtype)
Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
d_error

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
rewards

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
current_value

In [ ]:
current_rewards

In [ ]:
(current_rewards==0)*gamma*Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
rewards

In [ ]:
# current_rewards

In [ ]:
current_rewards[0]==life_cost

In [ ]:
current_rewards == -1 * life_cost

In [ ]:
current_rewards[0]

In [ ]:
choices = np.random.choice(time, batchsize)
current_action = actions[choices].unsqueeze(1)
current_rewards = rewards[choices].type(dtype)
indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
d_error = loss.clamp(-1, 1)
optimizer.zero_grad()
current_value.backward(d_error.data)
# optimizer.step()

In [ ]:
d_error.shape